In [2]:
from nustar_pysolar import planning, io
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [3]:
fname = io.download_occultation_times(outdir='../../data/')
print(fname)

../../data/NUSTAR.2019_008.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [4]:
tlefile = io.download_tle(outdir='../../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [5]:
tstart = '2019-01-12T00:00:00'
tend = '2019-01-12T23:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [6]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 356.9019354868723 deg


# Orbit 1: VLA Target:

Solar coordiantes [300, 195]



In [7]:
orbit1 = orbits[10].copy()
offset = [300, 195]*u.arcsec
midTime = (0.5*(orbit1[1] - orbit1[0]) + orbit1[0])
sky_pos = planning.get_sky_position(midTime, offset)
print("Orbit start: {} Orbit end: {}".format(orbit1[0].isoformat(), orbit1[1].isoformat()))
print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
print("")




Orbit start: 2019-01-12T16:33:30 Orbit end: 2019-01-12T17:35:30
Aim time: 2019-01-12T17:04:30 RA (deg): 293.60440062179936 deg Dec (deg): -21.601598633806983 deg



# Orbit 2: VLA Target:

Solar coordiantes [420, 195]



In [8]:
orbit2 = orbits[11]
offset = [420, 195]*u.arcsec
midTime = (0.5*(orbit2[1] - orbit2[0]) + orbit2[0])
sky_pos = planning.get_sky_position(midTime, offset)
print("Orbit start: {} Orbit end: {}".format(orbit2[0].isoformat(), orbit2[1].isoformat()))
print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
print("")




Orbit start: 2019-01-12T18:10:10 Orbit end: 2019-01-12T19:12:10
Aim time: 2019-01-12T18:41:10 RA (deg): 293.64122683714913 deg Dec (deg): -21.59271356238643 deg



# This is where you actually make the Mosaic for Orbit 3

In [9]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.

# Orbit3: 1946-2048
# Per Karl, SAA time 2031-2047
# Trim mosaic to 1946-2031:

orbit = orbits[12].copy()

orbit[1] = orbit[1].replace(minute=31)
print(orbit)
planning.make_mosaic(orbit, make_regions=True, outfile='orbit3_mosaic.txt', write_output=True)


[datetime.datetime(2019, 1, 12, 19, 46, 50), datetime.datetime(2019, 1, 12, 20, 31, 50)]
Orbit start: 2019-01-12T19:46:50 Orbit end: 2019-01-12T20:31:50
('Dwell per position:', 108.0)

NuSTAR Roll Angle to get roll relative to Sun of 0.00 is 356.51 deg
Step of FOV PA direction is 446.51 deg



# Now make the parking location for the end of Orbit 3:

In [10]:
tstart = orbit[1]
tend = orbits[12][1]
offset = [480, 195]*u.arcsec

midTime = (0.5*(tend - tstart)) + tstart
sky_pos = planning.get_sky_position(midTime, offset)
print("Orbit start: {} Orbit end: {}".format(tstart, tend))
print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
print("")





Orbit start: 2019-01-12 20:31:50 Orbit end: 2019-01-12 20:48:50
Aim time: 2019-01-12T20:40:20 RA (deg): 293.71284491161583 deg Dec (deg): -21.58027391140963 deg



# Mosiac for orbit 4

In [13]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.

# Orbit4: 2123-2225

# Per Karl, SAA time 2214-2230
# Trim mosaic to 2214-2230
orbit = orbits[13].copy()

orbit[1] = orbit[1].replace(minute=14)
print(orbit)
planning.make_mosaic(orbit, make_regions=True, outfile='orbit4_mosaic.txt', write_output=True)





[datetime.datetime(2019, 1, 12, 21, 23, 30), datetime.datetime(2019, 1, 12, 22, 14, 30)]
Orbit start: 2019-01-12T21:23:30 Orbit end: 2019-01-12T22:14:30
('Dwell per position:', 122.4)

NuSTAR Roll Angle to get roll relative to Sun of 0.00 is 356.48 deg
Step of FOV PA direction is 446.48 deg



# Now make the parking location for the end of Orbit 4:

Rotated orbit 2 by +80 arcsec

In [16]:
tstart = orbit[1]
tend = orbits[13][1]
offset = [520, 195]*u.arcsec

midTime = (0.5*(tend - tstart)) + tstart
sky_pos = planning.get_sky_position(midTime, offset)
print("Orbit start: {} Orbit end: {}".format(tstart, tend))
print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
print("")






Orbit start: 2019-01-12 22:14:30 Orbit end: 2019-01-12 22:25:30
Aim time: 2019-01-12T22:20:00 RA (deg): 293.77577004974904 deg Dec (deg): -21.569675994535743 deg

